In [31]:
import pandas as pd
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
data=pd.read_csv("Sorted.csv")

In [33]:
def load_test(data):
    X_test= data[data['Store Section'].isna()].cleanDescription
    return X_test

In [16]:
def load_data(path):
    return pd.read_csv(path)

In [34]:
def preprocessing(recipe):

    recipe = ''.join([i for i in recipe if not i.isdigit()])
    recipe= ''.join([i for i in recipe if i not in string.punctuation and i !="®" ])
    recipe= word_tokenize(recipe.lower())
    stemmer = PorterStemmer()
    recipe=" ".join([stemmer.stem(item) for item in recipe if item not in stopwords.words('english')])
    return recipe

In [35]:
data= load_data("Sorted.csv")
data.cleanDescription= data.cleanDescription.apply(preprocessing)

In [44]:
data.head()

,id,description,recipe_id,measure,parsedName,quantity,cleanDescription,Unnamed: 7,Store Section
0,1,1 cup cashews,2606,cup,cashews,1,cup cashew,NaN,Dry Goods
1,2,cold water to cover,2606,NaN,water,NaN,cold water cover,NaN,Household
2,3,1 cup coconut milk,2606,cup,coconut milk,1,cup coconut milk,NaN,Dry Goods
3,4,"4 tablespoons agave syrup, or to taste",2606,tablespoon,agave syrup,4,tablespoon agav syrup tast,NaN,Dry Goods
4,5,1 tablespoon vanilla extract,2606,tablespoon,vanilla extract,1,tablespoon vanilla extract,NaN,Dry Goods


In [36]:
df=data[['id','cleanDescription','Store Section']].dropna(axis=0)


In [37]:
X = df.cleanDescription
y=df['Store Section']

In [42]:
#--------------------------KNN-----------------------
knn=make_pipeline(TfidfVectorizer(),KNeighborsClassifier())
#knn.get_params().keys()
param_knn={'kneighborsclassifier__n_neighbors':range(1,5),
          }
grid_knn=GridSearchCV(knn,param_knn,cv=5)
grid_knn.fit(X,y)
knn=grid_knn.best_estimator_
print("best score knn {:.3f} avec les paramètres :{}".format(grid_knn.best_score_,grid_knn.best_params_))


best score knn 0.785 avec les paramètres :{'kneighborsclassifier__n_neighbors': 1}


c:\Users\gabri\Desktop\recipes\venv\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [43]:
knn.predict(load_test(data))

array(['Dry Goods', 'Dry Goods', 'Dry Goods', ..., 'Dry Goods',
       'Dry Goods', 'Dry Goods'], dtype=object)